In [12]:
import cv2
import mediapipe as mp
import pandas as pd
from datetime import datetime

# MediaPipe Hands solution setup
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Initialize Hands model
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5)

# Load video from specified path
cap = cv2.VideoCapture("Video/Walking_Man.mp4")  # Specify the video file

# Data storage for landmarks
landmark_data = []
frame_number = 0

# Loop through video frames to create landmark data
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_number += 1  # Increment frame number
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    results = hands.process(rgb_frame)

    # Initialize landmarks for the current frame
    landmarks_current_frame = [frame_number]

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            for idx, landmark in enumerate(hand_landmarks.landmark):
                if idx < 32:  # Limit to the first 32 landmarks
                    landmarks_current_frame.append(landmark.x)
                    landmarks_current_frame.append(landmark.y)

    landmark_data.append(landmarks_current_frame)

    # Show the frame
    cv2.imshow('MediaPipe Hands', frame)

    if cv2.waitKey(5) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()
cv2.destroyAllWindows()

# Save landmarks to CSV with current date and time
current_time = datetime.now().strftime("%Y%m%d_%H%M%S")  # Current date and time
csv_filename = f"{current_time}_landmarks.csv"  # Name for the CSV file

# Create DataFrame and save to CSV
column_names = ['frame_number'] + [f'landmark_{i+1}_x' for i in range(32)] + [f'landmark_{i+1}_y' for i in range(32)]
df = pd.DataFrame(landmark_data, columns=column_names)
df.to_csv(csv_filename, index=False)
print(f"Landmark positions saved to {csv_filename}")


FileNotFoundError: [Errno 2] No such file or directory: '20240926_041955_landmarks.csv'